# Transcript

In [6]:
class Transcript:
    def __init__(self, initialization_bytes: bytes):
        raise NotImplementedError("subclass responsibility")
        
    def append(self, bytes_to_append: bytes):
        raise NotImplementedError("subclass responsibility")
    
    def sample(self) -> bytes:
        raise NotImplementedError("subclass responsibility")


In [7]:
from hashlib import sha3_256

class Sha3_256Transcript(Transcript):
    def __init__(self, initialization_bytes: bytes):
        # """Creates a new SHA3-256 hasher. Initializes it with `initialization_bytes`"""
        # raise NotImplementedError("COMPLETE")
        self.hasher = sha3_256()
        self.hasher.update(initialization_bytes)
        
    def append(self, bytes_to_append: bytes):
        # """Updates the hasher with `bytes_to_append`"""
        # raise NotImplementedError("COMPLETE")
        self.hasher.update(bytes_to_append)
    
    def sample(self) -> bytes:
        # """
        # The return value is the digest of the hasher.
        # Replaces the hasher with a fresh new one and initialized
        # with the return value of this function
        # """
        # raise NotImplementedError("COMPLETE")
        sample =  self.hasher.digest()
        self.hasher = sha3_256()
        self.hasher.update(sample)
        return sample


In [8]:
# [TEST]
transcript = Sha3_256Transcript(bytes.fromhex("cafecafe"))
transcript.append(bytes.fromhex("cccc"))
transcript.sample()
transcript.append(bytes.fromhex("aaaa"))
transcript.append(bytes.fromhex("ffff"))
transcript.append(bytes.fromhex("eeee"))
assert(transcript.sample().hex() == "a54c90f1ffbd6c11954078e13d226174fa76ac7e8ebd041a6e26a67fff283bbb")
assert(transcript.sample().hex() == "f51b742ceda82332e600ad01f53d622210234062366d0fde02281753a3a57b2b")
assert(transcript.sample().hex() == "a72992c579ef9296a4f80e98a378fb2d7c95ab5aa3451b1afa822f9c3aa73a91")

### Schnorr Protocol for Discrete Log Knowledge Proof

1. **Setup:**
   - Alice generates a private key $a$ and computes her public key $A = g^a \text{mod }p$, where:
     - $p$ is a prime.
     - $g$ is a generator of the units group $\mathbb{F}_p^\times$.     
     
     
2. **Round 1**
   - Alice chooses a random value $r$ from $[0, p-1)$.
   - She computes $R = g^r \,(\text{mod } p)$.
   - Alice sends $R$ to Bob.


3. **Round 2:**
   - Bob chooses a challenge $b$ randomly from $[0, p-1)$.
   - Sends $b$ to Alice


4. **Round 3:**
   - Alice computes the response $s = r + ba \,(\text{mod }  p-1)$.
   - Alice sends $s$ to Bob.


5. **Verification:**
   - Bob checks if $R \cdot A^b$ equals $g^s$.
   - If true, the verification is successful, and Alice proves she knows $a$.


6. **Security Consideration:**
   - To prevent replay attacks, Alice and Bob should use fresh nonces for each proof.
   - The protocol is secure under the discrete logarithm assumption.


#### Diagram

| Step  | Alice                                          | Bob                                            |
|-------|------------------------------------------------|------------------------------------------------|
| S1    |Chooses random $r$                        |                                                |
|       |Computes $R = g^r$                                                 |                             |
|       |Sends $R$ to Bob                                                |                              |
| S2      |                                                |                       Chooses challenge $b$     |
|       |                                                |                       Sends $b$ to Alice         |
| S3  |    Computes $s = r + ba$ (mod $p-1$)      |                    |
|       |    Sends `s` to Bob                                             |                             |
| S4    |                   |   Checks $R\cdot A^b$ equals $g^s$                                             |
|       |           |      If true, verification successful                                          |
|       |                                                |                                                |


**Important: In what follows use the field `F` defined in the module `zk_adventures_types`**

In [18]:
import sys
sys.path.append("/home/sage/zk-adventures/exercises")

from zk_adventures_types import F, PRIME

In [68]:
from dataclasses import dataclass
from random import randint
 
class Party:
    def __init__(self, generator):
        self._generator = generator   

@dataclass
class Proof:
    R: int
    s: int
        
class SchnorrProver(Party):
    @staticmethod
    def simulate_send_field_element(element: int, transcript: Transcript):
        # """ Appends the element's big endian representation to the transcript """
        # raise NotImplementedError("COMPLETE")
        transcript.append(element.to_bytes((element.bit_length() + 7) // 8, 'big'))

    def prove(self, a: int, transcript: Transcript):
        r = randint(0, PRIME-1)
        R = self._generator ** r
        # Fiat-Shamir simulated interactions
        self.simulate_send_field_element(int(R), transcript)
        b = SchnorrVerifier.simulate_choose_challenge(transcript)
        
        s = (r + b * a) % (PRIME - 1)
        return Proof(R=R, s=s)
        
class SchnorrVerifier(Party):
    @staticmethod
    def simulate_choose_challenge(transcript: Transcript):
        # """
        # Samples bytes from the transcript.
        # Intereprets those bytes as an integer in big endian.
        # Returns that integer modulo `prime - 1`
        # """
        # raise NotImplementedError("COMPLETE")
        sample = transcript.sample()
        sample_int =  int.from_bytes(sample, byteorder='big')
        return sample_int % (PRIME - 1)

    
    def verify(self, A: int, proof: Proof, transcript: Transcript):
        # Fiat-Shamir simulated interactions
        SchnorrProver.simulate_send_field_element(int(proof.R), transcript)
        b = self.simulate_choose_challenge(transcript)
        
        R = F(proof.R)
        Ab = F(A**b)
        S = proof.s
         
        
        
        left_hand_side = R*Ab
        right_hand_side =  self._generator ** S
        print(left_hand_side, right_hand_side )
        return left_hand_side == right_hand_side

In [70]:
# [TEST]

generator = F(3)
assert generator.multiplicative_order() == F.order() - 1
a = 0xcafe
A = generator ** a

# Transcript initialization nonce
nonce = bytes.fromhex("deadbeef") + int(A).to_bytes(3, "big")

prover = SchnorrProver(generator)
proof = prover.prove(a, Sha3_256Transcript(nonce))

verifier = SchnorrVerifier(generator)
assert verifier.verify(A, proof, Sha3_256Transcript(nonce))

53837 53837
